In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [2]:
%cd
from DeepLearning.Project2.data_loading_preparation import load_audio_dataloaders_validation

/home/kacper


In [3]:
def get_generic_classifier(input_size, hidden_size, output_size):
    classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size),
    )
    return classifier
    
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cuda'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs)
            pred = torch.argmax(yhat, dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [5]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [6]:
def run_experiment(experiment_name, train_func, run, train_params=None):
    path = f"experiments_rnn/{experiment_name}_run_{run}_"
    print("Running experiment for ", path[:-1])
    
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(train_params)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": experiment_name,
                "train_params": train_params,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [7]:
training_device = "cuda"
device = "cuda"
max_epochs = 250

def oneLSTM(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=1024)
    criterion_weights = torch.tensor([1.] * 11 + [0.1]).to(device)
    model = LSTMSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


In [8]:
param_grid = {
    "small": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "smalldouble": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "tiny": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 1,
    },
    "tinydouble": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 2,
    },
    "wide": {
        "hidden_size": 80,
        "input_size": 20,
        "num_layers": 1,
    },
    "medium": {
        "hidden_size": 128,
        "input_size": 20,
        "num_layers": 1,
    },
}


experiment_list = [
    (
        f"trainer_{trainer.__name__}_params_{param_name}", 
        trainer,
        str(run),
        param
    )
    for run in range(11, 21) 
    for trainer in [oneLSTM]
    for param_name, param in param_grid.items()
]

In [9]:
len(experiment_list)

60

In [ ]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        "Experiment:", experiment[0]
    )
    try:
        run_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-04-22T21:53:43.078960 Experiment: trainer_oneLSTM_params_small
Running experiment for  experiments_rnn/trainer_oneLSTM_params_small_run_11
Report exists already for experiments_rnn/trainer_oneLSTM_params_small_run_11. Skipping...
Time: 2023-04-22T21:53:43.079134 Experiment: trainer_oneLSTM_params_smalldouble
Running experiment for  experiments_rnn/trainer_oneLSTM_params_smalldouble_run_11
Epoch: 1, Accuracy on validation set: 0.12673648215430647
Epoch: 2, Accuracy on validation set: 0.16969437914084207
Epoch: 3, Accuracy on validation set: 0.20880530027783714
Epoch: 4, Accuracy on validation set: 0.24364180380423167
Epoch: 5, Accuracy on validation set: 0.25753366103868347
Epoch: 6, Accuracy on validation set: 0.27869202821115624
Epoch: 7, Accuracy on validation set: 0.2876683051934174
Epoch: 8, Accuracy on validation set: 0.300064116264159
Epoch: 9, Accuracy on validation set: 0.31181876469331055
Epoch: 10, Accuracy on validation set: 0.3222910878392819
Epoch: 11, Accuracy 

Epoch: 135, Accuracy on validation set: 0.6886086770677495
Epoch: 136, Accuracy on validation set: 0.6796324000854883
Epoch: 137, Accuracy on validation set: 0.6873263517845694
Epoch: 138, Accuracy on validation set: 0.6877537935456294
Epoch: 139, Accuracy on validation set: 0.6753579824748878
Epoch: 140, Accuracy on validation set: 0.6941654199615303
Epoch: 141, Accuracy on validation set: 0.6930968155588801
Epoch: 142, Accuracy on validation set: 0.6841205385766189
Epoch: 143, Accuracy on validation set: 0.6807010044881385
Epoch: 144, Accuracy on validation set: 0.6971575122889506
Epoch: 145, Accuracy on validation set: 0.6905321649925198
Epoch: 146, Accuracy on validation set: 0.7014319298995512
Epoch: 147, Accuracy on validation set: 0.6954477452447104
Epoch: 148, Accuracy on validation set: 0.6973712331694807
Epoch: 149, Accuracy on validation set: 0.6971575122889506
Epoch: 150, Accuracy on validation set: 0.6971575122889506
Epoch: 151, Accuracy on validation set: 0.69544774524471

Epoch: 60, Accuracy on validation set: 0.5859208847299021
Epoch: 61, Accuracy on validation set: 0.5980433857932794
Epoch: 62, Accuracy on validation set: 0.5910250957039558
Epoch: 63, Accuracy on validation set: 0.5899617184176946
Epoch: 64, Accuracy on validation set: 0.5878349638451723
Epoch: 65, Accuracy on validation set: 0.6091025095703956
Epoch: 66, Accuracy on validation set: 0.6114419396001701
Epoch: 67, Accuracy on validation set: 0.6080391322841344
Epoch: 68, Accuracy on validation set: 0.584857507443641
Epoch: 69, Accuracy on validation set: 0.5965546575925138
Epoch: 70, Accuracy on validation set: 0.6197362824330073
Epoch: 71, Accuracy on validation set: 0.6010208421948107
Epoch: 72, Accuracy on validation set: 0.6225010633772863
Epoch: 73, Accuracy on validation set: 0.6071884304551255
Epoch: 74, Accuracy on validation set: 0.6152700978307103
Epoch: 75, Accuracy on validation set: 0.6186729051467461
Epoch: 76, Accuracy on validation set: 0.6252658443215653
Epoch: 77, Accu

Epoch: 38, Accuracy on validation set: 0.5539246968730057
Epoch: 39, Accuracy on validation set: 0.5649861731546479
Epoch: 40, Accuracy on validation set: 0.5552010210593491
Epoch: 41, Accuracy on validation set: 0.5607317592001702
Epoch: 42, Accuracy on validation set: 0.5898744947883429
Epoch: 43, Accuracy on validation set: 0.5813656668793874
Epoch: 44, Accuracy on validation set: 0.583918315252074
Epoch: 45, Accuracy on validation set: 0.5786002978089768
Epoch: 46, Accuracy on validation set: 0.5966815571155073
Epoch: 47, Accuracy on validation set: 0.5966815571155073
Epoch: 48, Accuracy on validation set: 0.5892363326951713
Epoch: 49, Accuracy on validation set: 0.5892363326951713
Epoch: 50, Accuracy on validation set: 0.6002978089768134
Epoch: 51, Accuracy on validation set: 0.6024250159540523
Epoch: 52, Accuracy on validation set: 0.6092320782812167
Epoch: 53, Accuracy on validation set: 0.624122527121889
Epoch: 54, Accuracy on validation set: 0.6009359710699851
Epoch: 55, Accur

Epoch: 179, Accuracy on validation set: 0.7083599234205488
Epoch: 180, Accuracy on validation set: 0.7192086790044672
Epoch: 181, Accuracy on validation set: 0.6936821952776006
Epoch: 182, Accuracy on validation set: 0.7049563922569666
Epoch: 183, Accuracy on validation set: 0.7102744097000638
Epoch: 184, Accuracy on validation set: 0.7149542650499894
Epoch: 185, Accuracy on validation set: 0.7162305892363326
Epoch: 186, Accuracy on validation set: 0.7202722824930866
Epoch: 187, Accuracy on validation set: 0.7066581578387577
Epoch: 188, Accuracy on validation set: 0.7177196341204
Epoch: 189, Accuracy on validation set: 0.6977238885343544
Epoch: 190, Accuracy on validation set: 0.7226122101680493
Epoch: 191, Accuracy on validation set: 0.7072963199319294
Epoch: 192, Accuracy on validation set: 0.7009146990002127
Epoch: 193, Accuracy on validation set: 0.7143161029568177
Epoch: 194, Accuracy on validation set: 0.7064454371410338
Epoch: 195, Accuracy on validation set: 0.7177196341204
Epo

...

Epoch: 137, Accuracy on validation set: 0.6723952738990333
Epoch: 138, Accuracy on validation set: 0.6680988184747583
Epoch: 139, Accuracy on validation set: 0.668313641245972
Epoch: 140, Accuracy on validation set: 0.6736842105263158
Epoch: 141, Accuracy on validation set: 0.6642320085929109
Epoch: 142, Accuracy on validation set: 0.6756176154672395
Epoch: 143, Accuracy on validation set: 0.6775510204081633
Epoch: 144, Accuracy on validation set: 0.6659505907626209
Epoch: 145, Accuracy on validation set: 0.666595059076262
Epoch: 146, Accuracy on validation set: 0.6745435016111708
Epoch: 147, Accuracy on validation set: 0.6693877551020408
Epoch: 148, Accuracy on validation set: 0.6657357679914071
Epoch: 149, Accuracy on validation set: 0.6702470461868958
Epoch: 150, Accuracy on validation set: 0.6685284640171858
Early stop
Time: 2023-04-23T01:48:04.664454 Experiment: trainer_oneLSTM_params_tinydouble
Running experiment for  experiments_rnn/trainer_oneLSTM_params_tinydouble_run_13
Epoch

Epoch: 125, Accuracy on validation set: 0.6783972864108544
Epoch: 126, Accuracy on validation set: 0.6870892516429934
Epoch: 127, Accuracy on validation set: 0.6847572609709561
Epoch: 128, Accuracy on validation set: 0.6813652745389018
Epoch: 129, Accuracy on validation set: 0.6796692813228747
Epoch: 130, Accuracy on validation set: 0.6824252702989188
Epoch: 131, Accuracy on validation set: 0.6758532965868137
Epoch: 132, Accuracy on validation set: 0.683061267754929
Epoch: 133, Accuracy on validation set: 0.6811532753868984
Epoch: 134, Accuracy on validation set: 0.6608013567945729
Epoch: 135, Accuracy on validation set: 0.6847572609709561
Epoch: 136, Accuracy on validation set: 0.6898452406190375
Epoch: 137, Accuracy on validation set: 0.685817256730973
Epoch: 138, Accuracy on validation set: 0.6955692177231291
Epoch: 139, Accuracy on validation set: 0.7017171931312275
Epoch: 140, Accuracy on validation set: 0.7038371846512614
Epoch: 141, Accuracy on validation set: 0.7101971592113632

Epoch: 78, Accuracy on validation set: 0.6934906256583105
Epoch: 79, Accuracy on validation set: 0.6960185380240151
Epoch: 80, Accuracy on validation set: 0.6974931535706762
Epoch: 81, Accuracy on validation set: 0.6977038129344849
Epoch: 82, Accuracy on validation set: 0.684853591742153
Epoch: 83, Accuracy on validation set: 0.6966505161154414
Epoch: 84, Accuracy on validation set: 0.6966505161154414
Epoch: 85, Accuracy on validation set: 0.6913840320202232
Epoch: 86, Accuracy on validation set: 0.677269854645039
Epoch: 87, Accuracy on validation set: 0.712028649673478
Epoch: 88, Accuracy on validation set: 0.7057088687592163
Epoch: 89, Accuracy on validation set: 0.6922266694754582
Epoch: 90, Accuracy on validation set: 0.6943332631135454
Epoch: 91, Accuracy on validation set: 0.7008637033916157
Epoch: 92, Accuracy on validation set: 0.7061301874868338
Epoch: 93, Accuracy on validation set: 0.7014956814830419
Epoch: 94, Accuracy on validation set: 0.7033916157573205
Epoch: 95, Accura

Epoch: 101, Accuracy on validation set: 0.7436769394261424
Epoch: 102, Accuracy on validation set: 0.7071200850159405
Epoch: 103, Accuracy on validation set: 0.7168969181721573
Epoch: 104, Accuracy on validation set: 0.7175345377258235
Epoch: 105, Accuracy on validation set: 0.7154091392136025
Epoch: 106, Accuracy on validation set: 0.7328374070138151
Epoch: 107, Accuracy on validation set: 0.7298618490967056
Epoch: 108, Accuracy on validation set: 0.7175345377258235
Epoch: 109, Accuracy on validation set: 0.7230605738575983
Epoch: 110, Accuracy on validation set: 0.7349628055260361
Epoch: 111, Accuracy on validation set: 0.7540913921360255
Epoch: 112, Accuracy on validation set: 0.7290116896918172
Epoch: 113, Accuracy on validation set: 0.7328374070138151
Epoch: 114, Accuracy on validation set: 0.7421891604675876
Epoch: 115, Accuracy on validation set: 0.7404888416578108
Epoch: 116, Accuracy on validation set: 0.7421891604675876
Epoch: 117, Accuracy on validation set: 0.72709883103081

Epoch: 109, Accuracy on validation set: 0.6857758620689656
Epoch: 110, Accuracy on validation set: 0.68125
Epoch: 111, Accuracy on validation set: 0.688146551724138
Epoch: 112, Accuracy on validation set: 0.6879310344827586
Epoch: 113, Accuracy on validation set: 0.6900862068965518
Epoch: 114, Accuracy on validation set: 0.6946120689655172
Epoch: 115, Accuracy on validation set: 0.6924568965517242
Epoch: 116, Accuracy on validation set: 0.6924568965517242
Epoch: 117, Accuracy on validation set: 0.6810344827586207
Epoch: 118, Accuracy on validation set: 0.6793103448275862
Epoch: 119, Accuracy on validation set: 0.6823275862068966
Epoch: 120, Accuracy on validation set: 0.6956896551724138
Epoch: 121, Accuracy on validation set: 0.696551724137931
Epoch: 122, Accuracy on validation set: 0.6853448275862069
Epoch: 123, Accuracy on validation set: 0.7030172413793103
Epoch: 124, Accuracy on validation set: 0.6954741379310345
Epoch: 125, Accuracy on validation set: 0.6926724137931034
Epoch: 126